# Creating Data Quality Measures

In [41]:
import numpy as np
import pandas as pd
import re
from collections import Counter
from matplotlib import pyplot as plt
import plotly.express as px
import scipy
from datetime import datetime
import joblib

import seaborn as sns
import math
from textdistance import damerau_levenshtein,jaro_winkler,jaccard,overlap,lcsseq,lcsstr,ratcliff_obershelp,levenshtein
import json
from outliertree import OutlierTree

In [42]:
silo1 = pd.read_csv(
    "/Users/joaoalmeida/Desktop/tese_local/chvng/CHVNGE_OBSCARE_WCR.csv", sep=";"
)
silo1["silo"]="CHVNGE"


In [43]:
silo2 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHEDV/CHEDV_OBSCARE_WCR_.xlsx",
    engine="openpyxl",
)
silo2["silo"]="CHEDV"

In [44]:
silo3 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHBV/CHBV_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo4 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHLN/CHLN_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo5 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHSJ/CHSJ_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo6 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/HSO/HSO_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo7 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/ULSAM/ULSAM_OBSCARE_WCR.xls"
)

silo8 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/ULSM/ULSM_OBSCARE_WCR_V1.xlsx",
    engine="openpyxl",
)

silo9 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHTS/CHTS_OBSCARE_WCR_.xlsx",
    engine="openpyxl",
)
silo3["silo"]="CHBV"
silo4["silo"]="CHLN"
silo5["silo"]="CHSJ"
silo6["silo"]="HSO"
silo7["silo"]="ULSAM"
silo8["silo"]="ULSM"
silo9["silo"]="CHTS"

WARNING *** file size (5079264) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [45]:
full_data = pd.concat([silo1,silo2,silo3,silo4,silo5,silo6,silo7,silo8,silo9]).reset_index(drop=True)

In [46]:
full_data

,IDENTIFICADOR,DATA_PARTO,IDADE_MATERNA,GS,PESO_INICIAL,PESO_ADMISSAO_INTERNAMENTO,IMC,CIGARROS,ALCOOL,ESTUPEFACIENTES,...,HEMATOLOGICA,RESPIRATORIA,CEREBRAL,CARDIACA,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON,silo
0,8 058,12/07/2018,34.0,"O,RH_NEGATIVO",62.0,NaN,27.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,28.0,10,CHVNGE
1,8 058,12/07/2018,34.0,"O,RH_NEGATIVO",62.0,NaN,27.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,28.0,10,CHVNGE
2,5 141,27/01/2017,24.0,"O,RH_POSITIVO",52.0,NaN,22.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3,CHVNGE
3,5 141,27/01/2017,24.0,"O,RH_POSITIVO",52.0,NaN,22.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3,CHVNGE
4,10 958,20/07/2020,36.0,"B,RH_POSITIVO",65.0,NaN,24.7,SIM,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,37.0,3,CHVNGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80869,7082,29/06/2017,27.0,"O,RH_POSITIVO",64.0,NaN,22.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.4,1,CHTS
80870,6579,20/04/2017,28.0,"A,RH_POSITIVO",69.0,NaN,25.7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Indução com dinoprostona,38.4,4,CHTS
80871,5070,27/12/2016,31.0,"B,RH_POSITIVO",48.0,NaN,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,39.3,1,CHTS
80872,5070,27/12/2016,31.0,"B,RH_POSITIVO",48.0,NaN,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,39.3,1,CHTS


In [47]:
#list(full_data["G_TERAPEUTICA"].unique())
#transform all from SIM/NAO to S/N e similar
#BACIA [nan 'ADEQUADA' 'LIMITE' 'INADEQUADA' 'A' 'L']
standard_map={"INADEQUADA":"Inadequada","L":"Limite","A":"Adequada","X":"Sim","S":"Sim","N":"Não",-7:np.nan,"ADEQUADA":"Adequada","LIMITE":"Limite","DESCONHECIDO,RH_DESCONHECIDO":np.nan,"DESCONHECIDO,":np.nan,"SIM":"Sim","NAO":"Não","-1":np.nan,-1:np.nan,"NS":np.nan,",":np.nan,"Sim":"Sim","UNKNOWN":np.nan,"Desconhecida":np.nan,"Desconhecido":np.nan,"DESCONHECIDO":np.nan," ":np.nan,"Desconhecido,":np.nan,"  ":np.nan}
def standardize_null(x,mapping):
    if x in mapping.keys():
        return mapping[x]
    if pd.isna(x):
        return np.nan
    return x

for col in full_data.columns:
    full_data[col]=full_data[col].apply(standardize_null,mapping=standard_map)

In [48]:
### Mapping
#transform into labels human readable
label_map={"APRESENTACAO_ADMISSAO":{"apr.cefala.3":"Cefálica","apr.pelv.1":"Pélvica","apr.esp.1":"Espádua","apr.trans.3":"Transversa","apr.desc.1":np.nan,"apr.face.2":"Face"}}

full_data.replace(label_map,inplace=True)


In [51]:
## relabel apresentacao
map_cols=['APRESENTACAO_42', 'APRESENTACAO_41',
       'APRESENTACAO_40', 'APRESENTACAO_39', 'APRESENTACAO_38',
       'APRESENTACAO_37', 'APRESENTACAO_36', 'APRESENTACAO_35',
       'APRESENTACAO_34', 'APRESENTACAO_33', 'APRESENTACAO_32',
       'APRESENTACAO_31', 'APRESENTACAO_30', 'APRESENTACAO_29',
       'APRESENTACAO_28', 'APRESENTACAO_27', 'APRESENTACAO_26',
       'APRESENTACAO_25', 'APRESENTACAO_24']

mapping={"CHLN":{"10044":"pélvico modo nadegas",
                '10052': 'situação transversa',
                   '10080': 'cefálica',
                   '10078': 'cefálica',
                   '10061': 'pélvico modo pés',
                   '10064': 'pélvica completa',
                   '10066': 'variável',
                   '10006': 'situação oblíqua',
                   '10019': 'pélvica',
                   '10079': 'pélvica',
                   '10081': 'apresentação composta',
                   '10083': 'cefálica',
                   '10082': 'pélvica'},
        "CHBV":{'1': 'cefalica',
                   '3': 'cefalica, dorso anterior',
                   '10000': 'obliquo, dorso anterior',
                   '8': 'pelvica dorso-posterior',
                   '4': 'cefalica dorso-posterior',
                   '5': 'pélvica',
                   '7': 'pélvica dorso-anterior',
                   '9': 'transversa'},
        "HSO":{'1': 'cefálica',
                   '3': 'cefálica, dorso anterior',
                   '10000': 'indiferente',
                   '10002': 'cefálica - dorso dta',
                   '10003': 'cefálica-dorso esq',
                   '10004': 'obliquo',
                   '10005': 'pélvico modo pés',
                   '10006': 'instável',
                   '8': 'pélvica dorso-posterior',
                   '4': 'cefálica dorso-posterior',
                   '5': 'pélvica',
                   '7': 'pélvica dorso-anterior',
                   '9': 'transversa'},
        "CHSJ":{  '1': 'cefálica',
                   '3': 'cefálica, dorso anterior',
                   '7': 'pélvica dorso-anterior',
                   '8': 'pélvica dorso-posterior',
                   '10000': 'cefálica insinuada',
                   '10001': 'pélvica',
                   '10012': 'cefálica e deflectida',
                   '10038': 'oblíqua',
                   '10042': 'cefálica muito insinuada',
                   '10043': 'situação transversa'},
        "CHTS":{    '1': 'cefalica',
                   '3': 'cefalica, dorso anterior',
                   '10000': 'cefalica dta',
                   '10001': 'pélvica esq',
                   '10002': 'cefalica dta',
                   '10003': 'pélvica dta',
                   '10004': 'espadua',
                   '10005': 'pélvica',
                   '10006': 'instavel',
                   '4': 'cefalica dorso-posterior',
                   '5': 'pélvica',
                   '7': 'pélvica dorso-anterior',
                   '9': 'transversa'},
        "CHEDV":{      '1': 'cefálica',
                   '3': 'cefálica, dorso-anterior',
                   '4': 'cefálica dorso-posterior',
                   '5': 'pélvica',
                   '7': 'pélvica dorso-anterior',
                   '8': 'pélvica dorso-posterior',
                   '9': 'transversa',
                   '10001': 'instável',
                   '10002': 'posterior alta',
                   '10003': 'pelve modo pés',
                   '10004': 'pelve muito insinuada',
                   '10005': 'cefálico dorso à esquerda',
                   '10006': 'cefálico dorso à direita',
                   '10007': 'pélvica dorso à direita',
                   '10008': 'pélvica dorso à esquerda',
                   '10009': 'cefálica insinuada'},
        "CHVNGE":{ '1': 'cefálica',
                  '10022':'cefálico esquerdo',
'10026':'apresentação composta',
'10027':'Cefálica dorso anterior',
               '10000': 'cefálica dorso à direita',
               '10001': 'cefálica dorso à esquerda',
               '10002': 'situação transversa, polo cefálico à esquerda',
               '10003': 'pélvica',
               '10004': 'situação transversa, polo cefálico à direita',
               '10005': 'cefálica dorso à direita',
               '10006': 'cefálica',
               '10007': 'cefálica dorso à esquerda',
               '10008': 'pélvica dorso à direita',
               '10010': 'cefálica muito insinuada',
               '10011': 'pelve franca, dorso à direita',
               '10012': 'pelve franca, dorso à esquerda',
               '10013': 'situação oblíqua com polo cefálico no quadrante inf esq',
               '10014': 'situação oblíqua, com o pólo cefálico no quadr sup direito e pelve no QIE',
               '10015': 'situação oblíqua',
               '10016': 'pelve desdobrada',
               '10017': 'pelve franca com dorso anterior esquerdo',
               '10018': 'pelve, modo pés',
               '10019': 'pelve completa',
               '10020': 'cefálica muito insinuada',
               '10021': 'pelve modo pés',
               '10023': 'pelve franca com dorso anterior',
               '10024': 'pelve modo nádegas',
               '10025': 'situação oblíqua com polo cefálico no QSE',
               '3': 'cefálica dorso-anterior',
               '4': 'cefálica dorso-posterior',
               '7': 'pélvica dorso-anterior',
               '8': 'pélvica dorso-posterior',
               '9': 'transverso'},
         "ULSM":{'1': 'cefálica',
                 '10017':'Espadua',
                   '3': 'cefálica, dorso anterior',
                   '10000': 'pélvica, dorso à esquerda',
                   '10001': 'pélvica',
                   '10002': 'instável',
                   '10003': 'cefálica, dorso à direita',
                   '10004': 'cefálica, dorso à esquerda',
                   '10005': 'pélvica',
                   '10006': 'transversa',
                   '10008': 'pélvica',
                   '10009': 'pélvica modo pés',
                   '10010': 'cefálica dorso-direita',
                   '10011': 'cefálica dorso-esquerda',
                   '10014': 'pélvica dorso-direita',
                   '10015': 'cefálica muito-inusitada',
                   '10013': 'pélvica dorso-direita',
                   '8': 'pélvica dorso-posterior',
                   '4': 'cefálica dorso-posterior',
                   '5': 'pélvica',
                   '7': 'pélvica dorso-anterior',
                   '9': 'transversa',
                   '10012': 'cefálica e deflectida',
                   '10038': 'oblíqua',
                   '10043': 'situação transversa'},
         "ULSAM":{   '1': 'cefálica',
                  '3': 'cefálica, dorso anterior',
                  '4': 'cefálica dorso-posterior',
                  '5': 'pélvica',
                  '7': 'pélvica dorso-anterior',
                  '8': 'pélvica dorso-posterior',
                  '9': 'transversa',

                  '10000': 'Instável',
                  '10001': 'cefálica, dorso à direita, posterior',
                  '10002': 'cefálico dorso anterior esquerdo',
                  '10003': 'transversa, dorso superior',
                  '10004': 'cefálica, dorso à direita',
                  '10006': 'cefálica, dorso à esquerda',
                  '10007':'pélvica, dorso à direita',
                  '10008': 'Pélvica',
                 '10009':'situação transversa, dorso inferior',
                 '10010':'pelvica dorso á esquerda',
                 '10011':'variável',
                 '10012':'situação transversa'}
        }


#CHVNGE
#'10022','cefálico esquerdo',
#'10026','apresentação composta',
#'10027','Cefálica dorso anterior',
def mapping_apresentacao(x,map_cols=map_cols,mapping=mapping):
    silo=x["silo"]
    for map_col in map_cols:
       # print(x[map_col])
        #print(pd.isna(x[map_col]))
        if not pd.isna(x[map_col]):
         #   print(map_col)
          #  print(str(int(x[map_col])))
           # print(mapping[silo])
            try:
                x[map_col]=mapping[silo][str(int(x[map_col]))]
            except:
                print(silo,x[map_col])
                x[map_col]=np.nan
    return x
full_data=full_data.apply(mapping_apresentacao,axis=1)

In [52]:
### Mapping
#transform into labels human readable
l_label_map={"Espadua":"espadua","Cefálica dorso anterior":"cefálica dorso anterior","cefálica, dorso anterior":"cefálica dorso anterior","cefálico dorso à esquerda":"cefálica dorso à esquerda","cefálica, dorso anterior":"cefálica dorso anterior",
             "pelvica dorso-posterior":"pélvica dorso-posterior","cefalica dta":"cefálica direita",
             "cefálica, dorso anterior":"cefálica dorso anterior","pelve modo pés":"pélvica modo pés",
             "situação oblíqua":"oblíqua","cefálica dorso-posterior":"cefálica dorso posterior","cefálico dorso à esquerda":"cefálica dorso à esquerda",
             "cefálica, dorso anterior":"cefálica dorso anterior","cefálica, dorso à esquerda":"cefálica dorso à esquerda",
             "cefálica - dorso dta":"cefálica dorso à direita","cefálica dorso-esquerda":"cefálica dorso à esquerda",
             "cefálica dorso-direita":"cefálica dorso à direita","cefálica dorso-anterior":"cefálica dorso anterior",
             "cefalica, dorso anterior":"cefálica dorso anterior","cefalica dorso-posterior":"cefálica dorso posterior",
             "cefálica, dorso à direita":"cefálica dorso à direita","cefálico dorso à direita":"cefálica dorso à direita",
             "cefálica-dorso esq":"cefálica dorso à esquerda","cefálica, dorso-anterior":"cefálica dorso anterior",
             "pelve, modo pés":"pelve modo pés","pélvico modo pés":"pélvica modo pés","pélvica, dorso à direita":"pélvica dorso à direita",
             "Pélvica":"pélvica","Instável":"instável","transverso":"transversa","situação transversa":"transversa",
             "pélvica dorso-direita":"pélvica dorso à direita","instavel":"instável","obliquo":"oblíqua","cefalica":"cefálica"}
for x,v in l_label_map.items():
    print(x,"||||",v)
full_label_map={}
for col in map_cols:
    full_label_map[col]=l_label_map
full_data.replace(full_label_map,inplace=True)

Espadua |||| espadua
Cefálica dorso anterior |||| cefálica dorso anterior
cefálica, dorso anterior |||| cefálica dorso anterior
cefálico dorso à esquerda |||| cefálica dorso à esquerda
pelvica dorso-posterior |||| pélvica dorso-posterior
cefalica dta |||| cefálica direita
pelve modo pés |||| pélvica modo pés
situação oblíqua |||| oblíqua
cefálica dorso-posterior |||| cefálica dorso posterior
cefálica, dorso à esquerda |||| cefálica dorso à esquerda
cefálica - dorso dta |||| cefálica dorso à direita
cefálica dorso-esquerda |||| cefálica dorso à esquerda
cefálica dorso-direita |||| cefálica dorso à direita
cefálica dorso-anterior |||| cefálica dorso anterior
cefalica, dorso anterior |||| cefálica dorso anterior
cefalica dorso-posterior |||| cefálica dorso posterior
cefálica, dorso à direita |||| cefálica dorso à direita
cefálico dorso à direita |||| cefálica dorso à direita
cefálica-dorso esq |||| cefálica dorso à esquerda
cefálica, dorso-anterior |||| cefálica dorso anterior
pelve, modo

In [53]:
full_data

,IDENTIFICADOR,DATA_PARTO,IDADE_MATERNA,GS,PESO_INICIAL,PESO_ADMISSAO_INTERNAMENTO,IMC,CIGARROS,ALCOOL,ESTUPEFACIENTES,...,HEMATOLOGICA,RESPIRATORIA,CEREBRAL,CARDIACA,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON,silo
0,8 058,12/07/2018,34.0,"O,RH_NEGATIVO",62.0,NaN,27.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,28.0,10,CHVNGE
1,8 058,12/07/2018,34.0,"O,RH_NEGATIVO",62.0,NaN,27.9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Cesariana,Cefálica de vértice,Ausência de trabalho de parto,28.0,10,CHVNGE
2,5 141,27/01/2017,24.0,"O,RH_POSITIVO",52.0,NaN,22.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3,CHVNGE
3,5 141,27/01/2017,24.0,"O,RH_POSITIVO",52.0,NaN,22.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,38.0,3,CHVNGE
4,10 958,20/07/2020,36.0,"B,RH_POSITIVO",65.0,NaN,24.7,Sim,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,37.0,3,CHVNGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80869,7082,29/06/2017,27.0,"O,RH_POSITIVO",64.0,NaN,22.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,40.4,1,CHTS
80870,6579,20/04/2017,28.0,"A,RH_POSITIVO",69.0,NaN,25.7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto eutócico cefálico,Cefálica de vértice,Indução com dinoprostona,38.4,4,CHTS
80871,5070,27/12/2016,31.0,"B,RH_POSITIVO",48.0,NaN,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,39.3,1,CHTS
80872,5070,27/12/2016,31.0,"B,RH_POSITIVO",48.0,NaN,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Parto auxiliado por ventosa,Cefálica de vértice,Espontâneo,39.3,1,CHTS


In [54]:
#categoriess on GS return some errors, handle with this:
def separate_gs(x):
    GS=x["GS"]
    if not type(GS)==str:
        return np.nan,np.nan
    if len(GS.split(","))>2:
        print(GS)
        return "check","check"
    else:
        return GS.split(",")



full_data[['GS_ABO','GS_RH']] = full_data.apply(separate_gs,axis=1,result_type="expand")
full_data.drop(columns=["GS"],inplace=True)

In [55]:
### Mapping
#transform into labels human readable
label_map={"GS_ABO":{"":np.nan},"GS_RH":{"RH_DESCONHECIDO":np.nan}}
full_data.replace(label_map,inplace=True)

In [56]:
from outliertree import OutlierTree

### random data frame with an obvious outlier
nrows = 100
np.random.seed(1)

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
train,test=train_test_split(full_data,train_size=0.5)

In [59]:
### fit model
outliers_model = OutlierTree()
outliers_df = outliers_model.fit(train, outliers_print = 10, return_outliers = True)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/outliertree/__init__.py:179: UserWarning: Attempting to use more than 1 thread, but package was built without multi-threading support - see the project's GitHub page for more information.
  warnings.warn(msg_omp)
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/outliertree/__init__.py:524: UserWarning: Some numeric columns have less than 3 different values - head: Index(['ESTIMATIVA_PESO_ECO_42', 'APRESENTACAO_42', 'NUM_RN'], dtype='object')
  warnings.warn("Some numeric columns have less than 3 different values - head: " + str(df_num.columns[too_few_values][:3]))


Reporting top 10 outliers [out of 617 found]


row [61009] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [4098.00]
	distribution: 66.667% <= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]



row [38984] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [3717.00]
	distribution: 66.667% >= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]



row [48189] - suspicious column: [SEMANAS_GESTACAO_PARTO] - suspicious value: [58.00]
	distribution: 99.995% <= 43.60 - [mean: 38.88] - [sd: 1.96] - [norm. obs: 40359]



row [21177] - suspicious column: [SEMANAS_GESTACAO_PARTO] - suspicious value: [48.60]
	distribution: 99.995% <= 43.60 - [mean: 38.88] - [sd: 1.96] - [norm. obs: 40359]



row [21642] - suspicious column: [IDADE_GESTACIONAL_ADMISSAO] - suspicious value: [49.00]
	distribution: 99.995% <= 45.10 - [mean: 38.69] - [sd: 2.21] - [norm. obs: 39963]



row [3332] - suspicious column: [IDADE_GESTACIONAL_ADMISSAO] - suspicious value: [49.00]
	di

In [61]:
### find outliers in new data
new_outliers = outliers_model.predict(test)

### print outliers in readable format
outliers_model.print_outliers(new_outliers)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/outliertree/__init__.py:632: UserWarning: Some categorical/ordinal columns had new values that were not present in the training data. These values will be ignored. Columns head: IDE
  warnings.warn(warn_msg)


Reporting top 10 outliers [out of 1650 found]


row [41626] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [4254.00]
	distribution: 66.667% <= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]



row [40599] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [3343.00]
	distribution: 66.667% >= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]



row [39093] - suspicious column: [IMC] - suspicious value: [463.70]
	distribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]



row [67447] - suspicious column: [IMC] - suspicious value: [454.20]
	distribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]



row [5901] - suspicious column: [IMC] - suspicious value: [448.80]
	distribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]



row [58729] - suspicious column: [IMC] - suspicious value: [437.00]
	distribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 343

,suspicious_value,group_statistics,conditions,tree_depth,uses_NA_branch,outlier_score
58968,"{'column': 'ESTIMATIVA_PESO_ECO_38', 'value': ...","{'lower_thr': 2974.0, 'pct_above': 1.0, 'mean'...","[{'column': 'G_TERAPEUTICA', 'comparison': 'in...",3.0,False,9.606517e-03
20724,"{'column': 'NUMERO_CONSULTAS_PRE_NATAL', 'valu...","{'upper_thr': 31.0, 'pct_below': 1.0, 'mean': ...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",2.0,False,6.836937e-03
58729,"{'column': 'IMC', 'value': 437.0, 'decimals': 0}","{'upper_thr': 318.3, 'pct_below': 0.9999709243...",[],0.0,False,1.323704e-04
49398,"{'column': 'EUTOCITO_ANTERIOR', 'value': 2.0, ...","{'upper_thr': 1.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'CESARIANAS_ANTERIOR', 'comparison...",4.0,False,3.922715e-03
69729,"{'column': 'BISHOP_CONSISTENCIA', 'value': 2.0...","{'upper_thr': 0.0, 'pct_below': 0.957446808510...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",1.0,False,2.500000e-25
...,...,...,...,...,...,...
14655,"{'column': 'IDADE_MATERNA', 'value': 22.0, 'de...","{'lower_thr': 32.0, 'pct_above': 1.0, 'mean': ...","[{'column': 'TIPO_PARTO', 'comparison': 'in', ...",3.0,False,9.822345e-03
21890,"{'column': 'EUTOCITO_ANTERIOR', 'value': 2.0, ...","{'lower_thr': 3.0, 'pct_above': 1.0, 'mean': 3...","[{'column': 'IDENTIFICADOR', 'comparison': 'in...",4.0,False,1.000000e-24
40142,"{'column': 'EUTOCITO_ANTERIOR', 'value': 1.0, ...","{'upper_thr': 0.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",4.0,False,1.000000e-24
10557,"{'column': 'BISHOP_POSICAO', 'value': 1.0, 'de...","{'upper_thr': 0.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'G_TERAPEUTICA', 'comparison': 'in...",2.0,False,1.000000e-24


In [62]:
joblib.dump(outliers_model, 'gritbot.sav')

['gritbot.sav']

In [108]:
def create_response_outlier(out):
    results=[]
    #for _,x in out[out["suspicious_value"]!={}].iterrows():
   # susp=x["suspicious_value"]
    #print(susp)
    #print(susp["column"])
  #  cond=x["group_statistics"]
    #print(cond)
    #results.append(f"""Suspicious Column: {susp["column"]}. Suspicious Value: {susp["value"]}""")
    with Capturing() as output:
        outliers_model.print_outliers(out)
    
    #print('done')
    #print('output:', output)
       
    #out = f.getvalue()
    return output

In [109]:
from io import StringIO 
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

In [110]:
create_response_outlier(new_outliers)

done
output: ['Reporting top 10 outliers [out of 1650 found]', '', '', 'row [41626] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [4254.00]', '\tdistribution: 66.667% <= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]', '', '', '', 'row [40599] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [3343.00]', '\tdistribution: 66.667% >= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]', '', '', '', 'row [39093] - suspicious column: [IMC] - suspicious value: [463.70]', '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]', '', '', '', 'row [67447] - suspicious column: [IMC] - suspicious value: [454.20]', '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]', '', '', '', 'row [5901] - suspicious column: [IMC] - suspicious value: [448.80]', '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]', '', '', '', 'row [58729] - suspicious column: [IMC] - su

['Reporting top 10 outliers [out of 1650 found]',
 '',
 '',
 'row [41626] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [4254.00]',
 '\tdistribution: 66.667% <= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]',
 '',
 '',
 '',
 'row [40599] - suspicious column: [ESTIMATIVA_PESO_ECO_41] - suspicious value: [3343.00]',
 '\tdistribution: 66.667% >= 3787.00 - [mean: 3787.00] - [sd: nan] - [norm. obs: 1]',
 '',
 '',
 '',
 'row [39093] - suspicious column: [IMC] - suspicious value: [463.70]',
 '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]',
 '',
 '',
 '',
 'row [67447] - suspicious column: [IMC] - suspicious value: [454.20]',
 '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]',
 '',
 '',
 '',
 'row [5901] - suspicious column: [IMC] - suspicious value: [448.80]',
 '\tdistribution: 99.997% <= 318.30 - [mean: 25.06] - [sd: 5.76] - [norm. obs: 34392]',
 '',
 '',
 '',
 'row [58729] - suspicious col

In [87]:
new_outliers[new_outliers["suspicious_value"]!={}]

,suspicious_value,group_statistics,conditions,tree_depth,uses_NA_branch,outlier_score
58968,"{'column': 'ESTIMATIVA_PESO_ECO_38', 'value': ...","{'lower_thr': 2974.0, 'pct_above': 1.0, 'mean'...","[{'column': 'G_TERAPEUTICA', 'comparison': 'in...",3.0,False,9.606517e-03
20724,"{'column': 'NUMERO_CONSULTAS_PRE_NATAL', 'valu...","{'upper_thr': 31.0, 'pct_below': 1.0, 'mean': ...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",2.0,False,6.836937e-03
58729,"{'column': 'IMC', 'value': 437.0, 'decimals': 0}","{'upper_thr': 318.3, 'pct_below': 0.9999709243...",[],0.0,False,1.323704e-04
49398,"{'column': 'EUTOCITO_ANTERIOR', 'value': 2.0, ...","{'upper_thr': 1.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'CESARIANAS_ANTERIOR', 'comparison...",4.0,False,3.922715e-03
69729,"{'column': 'BISHOP_CONSISTENCIA', 'value': 2.0...","{'upper_thr': 0.0, 'pct_below': 0.957446808510...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",1.0,False,2.500000e-25
...,...,...,...,...,...,...
14655,"{'column': 'IDADE_MATERNA', 'value': 22.0, 'de...","{'lower_thr': 32.0, 'pct_above': 1.0, 'mean': ...","[{'column': 'TIPO_PARTO', 'comparison': 'in', ...",3.0,False,9.822345e-03
21890,"{'column': 'EUTOCITO_ANTERIOR', 'value': 2.0, ...","{'lower_thr': 3.0, 'pct_above': 1.0, 'mean': 3...","[{'column': 'IDENTIFICADOR', 'comparison': 'in...",4.0,False,1.000000e-24
40142,"{'column': 'EUTOCITO_ANTERIOR', 'value': 1.0, ...","{'upper_thr': 0.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'DATA_PARTO', 'comparison': 'in', ...",4.0,False,1.000000e-24
10557,"{'column': 'BISHOP_POSICAO', 'value': 1.0, 'de...","{'upper_thr': 0.0, 'pct_below': 1.0, 'mean': 0...","[{'column': 'G_TERAPEUTICA', 'comparison': 'in...",2.0,False,1.000000e-24
